In [ ]:
import torch
import alpaca_hotpot_qa
import transformers
from peft import PeftModel
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer



## Load Alpaca model

In [ ]:
if torch.cuda.is_available():
    device = "cuda"

In [ ]:
load_8bit = False
base_model = 'decapoda-research/llama-7b-hf'
lora_weights = 'tloen/alpaca-lora-7b'

tokenizer = LlamaTokenizer.from_pretrained(base_model)
if device == "cuda":
    alp_model = LlamaForCausalLM.from_pretrained(base_model, load_in_8bit=load_8bit,
                    torch_dtype=torch.float16, device_map="auto")

In [ ]:
#if can't find xxx.json error occurred, check the lora_weights variable, trying using full system path

if device == "cuda":
    alp_model = PeftModel.from_pretrained(alp_model, lora_weights, torch_dtype=torch.float16)

# unwind broken decapoda-research config
alp_model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
alp_model.config.bos_token_id = 1
alp_model.config.eos_token_id = 2

if not load_8bit:
    alp_model.half()  # seems to fix bugs for some users.

alp_model.eval()
if torch.__version__ >= "2" and sys.platform != "win32":
    alp_model = torch.compile(alp_model)

## Load phone database pickle file

(phone_dataset.pkl was a bad naming, it should have been phone_database.pkl instead, but it's too time-consuming to rerun everything that uses this file, so just keeping it as it is)

In [ ]:
import pickle
#load phonedb data
with open("phone_dataset.pkl", "rb") as f:
    pdb = pickle.load(f)
phonedb_data, name_map = pdb

## Import the Alpaca-Hotpot model (a.k.a Alpaca-Fusion model) 

In [ ]:
from alpaca_hotpot_qa import AlpacaHotPotQA

In [ ]:
alphot_qa_inference = AlpacaHotPotQA(device, alp_model, tokenizer, phonedb_data, name_map)

## Perform inference
Everything was wrapped in alpaca_hotpot_qa.py, simply pass the question as the function argument, the model inference method will return an answer, the default parameter for print_process is true, hence it will print the internal inference results of each sub-models in the Alpaca-fusioning with other classical NLP models

In [ ]:
question = '''\
"How do the camera capabilities of the Apple iPhone 12, Samsung Galaxy S21, and Xiaomi Mi 11 compare?"
'''
answer = alphot_qa_inference.inference(question)